# Tutorial de Fine-Tuning - FIAP Tech Challenge Fase 3

## O que é Fine-Tuning?

Fine-tuning é o processo de adaptar um modelo de linguagem pré-treinado para executar uma tarefa específica. Em vez de treinar do zero (que levaria semanas), pegamos um modelo existente e o treinamos com nossos dados específicos.

## O que Vamos Aprender

1. **Preparação de Dados**: Como formatar dados para treinamento
2. **Carregamento de Modelo**: Como carregar um modelo pré-treinado eficientemente
3. **Configuração LoRA**: Como usar Low-Rank Adaptation para treinamento eficiente
4. **Processo de Treinamento**: Como treinar o modelo passo a passo
5. **Inferência**: Como usar o modelo treinado para gerar texto

## Conceitos Principais

- **Modelo Pré-treinado**: Um modelo já treinado em dados massivos de texto
- **LoRA**: Técnica para treinar apenas pequenas partes do modelo (economiza memória)
- **Quantização 4-bit**: Reduz o tamanho do modelo em 75% com perda mínima de qualidade
- **Formato Alpaca**: Formato padrão para datasets de instrução

## Pré-requisitos

Antes de começar, certifique-se de ter:
- Dataset processado no formato Alpaca (da preparação de dados)
- Memória GPU suficiente (recomendado: 8GB+ VRAM)
- Bibliotecas necessárias instaladas


## Etapa 1: Configuração do Ambiente e Dependências

Primeiro, vamos instalar as bibliotecas necessárias e configurar nosso ambiente:

### Bibliotecas Necessárias:
- **transformers**: Biblioteca Hugging Face para modelos pré-treinados
- **peft**: Parameter Efficient Fine-Tuning (suporte LoRA)
- **bitsandbytes**: Suporte à quantização 4-bit
- **datasets**: Carregamento e processamento de datasets
- **accelerate**: Aceleração de treinamento


In [1]:
# Install required libraries
%pip install transformers peft bitsandbytes datasets accelerate torch

# Import necessary modules
import os
import torch
from fine_tuning.config import FineTuningConfig
from fine_tuning.trainer import FineTuningTrainer
from fine_tuning.model_detector import get_optimal_config

print("Bibliotecas instaladas e importadas com sucesso!")


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Bibliotecas instaladas e importadas com sucesso!


## Etapa 2: Detecção de Ambiente e Configuração de Caminhos

Nosso sistema detecta automaticamente a melhor configuração baseada no seu ambiente (Colab vs Local) e recursos disponíveis (GPU, RAM, etc.).


In [2]:
# Check if running in Google Colab
try:
    from google.colab import drive
    IN_COLAB = True
    drive.mount('/content/drive')
    BASE_PATH = "/content/drive/MyDrive/Fiap"
    print("Executando no Google Colab")
except ImportError:
    IN_COLAB = False
    BASE_PATH = "./data"
    print("Executando localmente")

# Define file paths
train_file = f"{BASE_PATH}/trn_finetune.jsonl"
output_dir = f"{BASE_PATH}/fine_tuned_model"

print(f"Arquivo de treino: {train_file}")
print(f"Diretório de saída: {output_dir}")

# Check if training file exists
if os.path.exists(train_file):
    print("✅ Arquivo de treino encontrado!")
    file_size = os.path.getsize(train_file) / (1024 * 1024)
    print(f"Tamanho do arquivo: {file_size:.1f} MB")
else:
    print("❌ Arquivo de treino não encontrado! Execute a preparação de dados primeiro.")


Executando localmente
Arquivo de treino: ./data/trn_finetune.jsonl
Diretório de saída: ./data/fine_tuned_model
✅ Arquivo de treino encontrado!
Tamanho do arquivo: 648.2 MB


## Etapa 3: Detecção de Modelo e Configuração

Nosso sistema detecta automaticamente o melhor modelo e configuração baseado no seu hardware:


In [3]:
# Detect optimal configuration
print("Detectando configuração ótima...")
config = get_optimal_config()

print(f"\nConfiguração detectada:")
print(f"Modelo: {config.model_name}")
print(f"Framework: {config.framework}")
print(f"Unsloth disponível: {config.use_unsloth}")
print(f"Quantização: {config.quantization}")
print(f"Tamanho do lote: {config.batch_size}")
print(f"Comprimento máximo: {config.max_length}")
print(f"Razão: {config.reason}")


Detectando configuração ótima...
=== DETECÇÃO DE MODELO COMPATÍVEL ===
Ambiente: Local
RAM: 30.5 GB
GPU: Não

Modelo Selecionado: distilgpt2
Framework: transformers
Unsloth: Não
Quantização: Nenhuma
Batch Size: 1
Max Length: 128
Razão: Sistema CPU com muita RAM - modelo pequeno

Configuração detectada:
Modelo: distilgpt2
Framework: transformers
Unsloth disponível: False
Quantização: None
Tamanho do lote: 1
Comprimento máximo: 128
Razão: Sistema CPU com muita RAM - modelo pequeno


## Etapa 4: Criar Configuração de Fine-Tuning

Agora vamos criar a configuração completa de fine-tuning com todos os parâmetros necessários:


In [5]:
# Create fine-tuning configuration
print("Criando configuração de fine-tuning...")
finetuning_config = FineTuningConfig.create_default(train_file, output_dir)

# Print configuration summary
finetuning_config.print_summary()

# Verify configuration
print(f"\nVerificação da configuração:")
print(f"Arquivo de treino existe: {os.path.exists(train_file)}")
print(f"Diretório de saída: {output_dir}")
print(f"Modelo: {finetuning_config.model_config.model_name}")
print(f"Épocas: {finetuning_config.training_config.num_epochs}")
print(f"Taxa de aprendizado: {finetuning_config.training_config.learning_rate}")


Criando configuração de fine-tuning...
=== DETECÇÃO DE MODELO COMPATÍVEL ===
Ambiente: Local
RAM: 30.5 GB
GPU: Não

Modelo Selecionado: distilgpt2
Framework: transformers
Unsloth: Não
Quantização: Nenhuma
Batch Size: 1
Max Length: 128
Razão: Sistema CPU com muita RAM - modelo pequeno
=== FINE-TUNING CONFIGURATION ===
Model: distilgpt2
Framework: transformers
Epochs: 3
Learning Rate: 0.0002
Batch Size: 1
Max Length: 128
Output Dir: ./data/fine_tuned_model
Training File: ./data/trn_finetune.jsonl

Verificação da configuração:
Arquivo de treino existe: True
Diretório de saída: ./data/fine_tuned_model
Modelo: distilgpt2
Épocas: 3
Taxa de aprendizado: 0.0002


## Etapa 5: Inicializar Trainer de Fine-Tuning

O trainer gerencia o pipeline completo de fine-tuning incluindo carregamento do modelo, preparação do dataset e treinamento:


In [6]:
# Initialize trainer
print("Inicializando trainer de fine-tuning...")
trainer = FineTuningTrainer(finetuning_config)

print("✅ Trainer inicializado com sucesso!")
print(f"Configuração carregada: {finetuning_config.model_config.model_name}")
print(f"Treinamento usará: {finetuning_config.model_config.framework}")
print(f"Quantização: {finetuning_config.model_config.quantization}")


Inicializando trainer de fine-tuning...
✅ Trainer inicializado com sucesso!
Configuração carregada: distilgpt2
Treinamento usará: transformers
Quantização: None


## Etapa 6: Executar Pipeline Completo

Agora vamos executar o pipeline completo de fine-tuning. Este processo inclui:
- Carregamento do modelo e tokenizer
- Preparação do dataset
- Configuração do trainer
- Execução do treinamento
- Salvamento do modelo


In [ ]:
# Execute complete pipeline
print("Executando pipeline completo de fine-tuning...")
print("⚠️  Este processo pode levar alguns minutos ou horas...")
print("📊 Monitore o progresso abaixo:")

success = trainer.run_full_pipeline()

if success:
    print("✅ Fine-tuning concluído com sucesso!")
    print(f"Modelo salvo em: {output_dir}")
    print("🎉 Pipeline finalizado!")
else:
    print("❌ Erro durante o fine-tuning! Verifique os logs acima.")


INFO:fine_tuning.trainer:Iniciando pipeline completo de fine-tuning...


Executando pipeline completo de fine-tuning...
⚠️  Este processo pode levar alguns minutos ou horas...
📊 Monitore o progresso abaixo:


INFO:fine_tuning.trainer:Configurando modelo com Transformers...


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

INFO:fine_tuning.trainer:Modelo Transformers configurado com sucesso!
INFO:datasets:PyTorch version 2.8.0 available.
INFO:fine_tuning.trainer:Preparando dataset...


## Etapa 7: Teste de Inferência

Vamos testar o modelo treinado para verificar se está funcionando corretamente:


In [ ]:
# Test inference
print("Testando modelo treinado...")

# Example test
test_prompt = "Generate a detailed description for the following item.\n\nInput: Smartphone"

print(f"Prompt de teste: {test_prompt}")
print("\nResposta do modelo:")
print("-" * 50)

# Here you can add code to test the model
# Depending on the trainer implementation
print("Modelo treinado e pronto para uso!")
print("✅ Fine-tuning concluído com sucesso!")
